# Argos training tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/subhacom/argos_tutorials/blob/main/ArgosTrainingTutorial.ipynb)

# Introduction

**Argos** is a toolkit for tracking multiple animals without markers in cluttered and possibly changing background. There are many excellent tools for multiple-animal tracking in controlled laboratory environments where the background and illumination can be made quite uniform and steady: for example, UMATracker (Yamanaka and Takeuchi, 2018), idtracker.ai (Romero-Ferrero et al., 2019), TRex (Walter and Couzin, 2021), FastTrack (Gallois and Candelier, 2021). Argos is specifically designed for more general situation, like in the field, where the number of animals, the illumination, and the background, can all be variable. It is aimed at behavioral neuroscientists and ethologists who are not necessarily programming experts. Instead of coming up with yet another Deep Neural Network for identifying and tracking objects, Argos puts together existing opensource networks and algorithms with a simple graphical interface. 

To track multiple objects with Argos, you have to 
 (1) train a neural network to detect your objects of interest using annotated images, 
 (2) record you videos (you can use Argos Capture tool) and optionally shrink them based on motion (so that parts with little or no movement are removed),
 (3) automatically track the objects using Argos Tracking tool,
 (4) check and correct the tracks using Argos review tool,
 (5) do further analysis according to your needs.

Argos tracking works in two steps, in each frame it detects and locates the objects of interest, and then matches them with the previous frame to obtain the trajectories. The object detection step uses a Convolutional Neural Network called YOLACT (Bolya, Daniel, Chong Zhou, Fanyi Xiao, and Yong Jae Lee. 2019. “YOLACT: Real-Time Instance Segmentation,” https://arxiv.org/abs/1904.02689v2). YOLACT authors provide pretrained netwroks to detect objects from the COCO dataset.  If your objects of interest are not in this dataset, you need to re-train a network to identify them. This step is much faster than training a CNN from scratch. The training requires a CUDA-capable GPU, but if you do not have one, it is possible to do this on the free Google Colab platform. Multiple categories of objects can be detected and tracked simultaneously, but for simplicity from now on we shall assume a single object class of interest.

This tutorial will walk you through steps to train the YOLACT network using your own sample images on Google Colab.

## Prepare training data
The training data consists of images along with annotation. Collect many pictures of your object type of interest, something like a hundred should be good. Having them photographed in diverse backgrounds and from many different angles helps the network learn to detect them in more general situations. Once you have collected the images, put them in a folder, say `training_images`.

The annotation consists of the outline of each object of the class of interest. The training program expects this information to be in a specific format, `COCO` format (see https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch for a readable description of it). Although you can create the annotation using several tools (a classic one is [labelme](http://labelme.csail.mit.edu/Release3.0/) and a nice [standalone implementation](https://github.com/wkentaro/labelme)), Argos comes with an **Annotation** tool specifically for helping with the training set. See the [documentation](https://argos.readthedocs.io/en/latest/user.html#annotate-generate-training-data-for-yolact) for details on using this tool.

Export the annotated training data into a different folder, say `training_data`. The Argos Annotation tool should create two folders, `training` and `validation` and a configuration file with `.yaml` extension. The last one is just a text file, and you should open and edit it to fit your requirements. As an example, a configuration file may contain:

```
base: yolact_base_config
dataset:
  class_names:
  - babylocust
  has_gt: true
  name: null
  train_images: C:/Users/me/Documents/training_data/training
  train_info: C:/Users/me/Documents/training_data/training/annotations.json
  valid_images: C:/Users/me/Documents/training_data/validation
  valid_info: C:/Users/me/Documents/training_data/validation/annotations.json
lr_steps:
- 28000
- 60000
- 70000
- 80000
- 100000
- 160000
- 200000
max_iter: 240000
max_size: 550
name: babylocust_weights
num_classes: 2
```

- `base` attributes tells YOLACT that this configuration is derived from the one named `yolact_base_config`, which has certain default properties so we do not have to specify too many things.  This is set by the Annotation tool.
- `class_names` lists the classes of objects to detect. In this case we have a single class: `babylocust`. This is set in the Annotation tool.
- The last four attributes under `dataset` are important for the training. `train_images` specifies the path to the folder where the training images The annotation tool puts the full path on the local machine: ` C:/Users/me/Documents/training_data/training`. This must be modified to the correct path on Colab. `train_info` field specifies the annotation filename, which is called `annotations.json` and is located inside `training` folder. Similarly `valid_images` and `valid_info` specify the paths for validation images and annotation, which must be modified for Colab.
- `lr_steps` specifies after how many steps the learning rate (a training parameter) should be updated. A standard approach is to start with a large learning rate, so the network undergoes big changes at first, but then reduce it gradually as training progresses. The above example tells the YOLACT to update the learning rate after 28000, 60000, ..., 200000 steps. You need to modify this based on how many iterations in total you want, and how you want the learning rate to change over that.
- `max_iter` specifies the total number of training iterations, more iterations improve learning, but also takes more time. Expect something like a day or two for 20,000 iterations with about training 150 images (# of images exported by Annotation tool, which actually splits up each high-resolution image into smaller pieces).
- `max_size` is the size of the training images, here 550x550 pixels. This is set by the Annotation tool.
- `name` specifies the name of the file storing the network weights.  This is set in the Annotation tool.
- `num_classes` is the total number of object classes, which is 1 more than the number of `class_names` specified, where the unnamed class is background (everything except objects of the class of interest). This is set by the Annotation tool.


Create a directory called `weights` inside `training_data` to store the final, and if you wish, intermediate weights (saving intermediate weights is essential on Google Colab, so that you can restart training from the last saved weights). Download a suitable backbone network linked in the YOLACT paged, e.g., [this](https://drive.google.com/file/d/1tvqFPd4bJtakOlmn-uIA492g2qurRChj/view?usp=sharing) and copy it into this directory.





## Upload your data
Google Colab provides temporary local harddisk space, but it is lost along with all the data once you are disconnected. Since the training takes many hours, you need a more persistent storage. Google Drive is ideal for this.

You can upload (or zip and upload) the `training_data` folder to your Google Drive and mount the drive using Colab menu. On the left pane in Colab, click the folder icon (Files) to display a filesystem tree. The toolbar on top of this has a drive icon, click it to mount your Google Drive. The path to the base directory of your Google Drive after mounting is `/content/drive/MyDrive`.

## Edit the training configuration
Now navigate in the directory tree in the left pane to locate the uploaded folder, and double click the configuration file. This will open it on the right pane for editing. Modify the paths here, e.g., `/content/drive/MyDrive/training_data/locust_config.yaml`. You can also use relative path, it must be relative to the location where you will run the training command.

## Set up runtime environment

You need a GPU to run the training program. By default Google gives you a CPU. To switch the runtime to a GPU, Click Runtime-> Change Runtime, and in the dialog, select `GPU` from the `Hardware accelerator` menu.



Now install Argos toolkit with the `pip` command:

In [ ]:
!pip install -U argos-toolkit

Now go to training directory

In [ ]:
TRAINING_DIR = '/content/drive/MyDrive/training_dir'
%cd {TRAINING_DIR}

## Start training 
If this is the first time you are running the training, run the following:

In [ ]:
weights = os.listdir('./weights')
assert len(weights) > 0

if len(weights) == 1:
  !python -m yolact.train --config locust_config.yaml --validation_epoch 20 --save_interval 1000
else:
  fno = []
  for fname in weights:
    parts = fname.rpartition('.')[0].split('_')
    if len(parts) > 2:
      try:
        epoch = int(parts[-2])
        batch = int(parts[-1])
      except ValueError:
        if parts[-1] == 'interrupt':
          epoch = int(parts[-3])
          batch = int(parts[-2])
      fno.append((fname, epoch, batch))
  fno = sorted(fno, key=operator.itemgetter(2))
  fname, epoch, batch = fno[-1]
  print('Resuming', fname)
  !python -m yolact.train --config locust_config.yaml --validation_epoch 20 --cuda=True --save_interval 1000 --resume=weights/{fname} --start_iter=-1

The first line above collects the names of all the files inside the `weights` directory. There must be at least one (the initial backbone) file there. If this is the first time you are running this, there will be just this one file, in which case the command `python -m yolact.train --config locust_config.yaml --validation_epoch 20 --save_interval 1000` will be executed. For `yolact_base_config`, the default weights filename is `resnet101_reducedfc.pth`, which we have copied in the `weights` directory.

The `--save_interval 1000` tells YOLACT to save the network weights after every 1000 training iterations. 

It is likely that you will get disconnected for inactivity after a few thousand iterations, and when you get back to it, you can start with the last saved weights. That is what the `else` clause above is doing. It checks the filenames in the `weights` directory to find the latest weights file (variable `fname`), and runs the command `python -m yolact.train --config locust_config.yaml --validation_epoch 20 --cuda=True --save_interval 1000 --resume=weights/{fname} --start_iter=-1`, which will start the training from after the iteration in which this weights file had been saved.

## Download the trained weights and configuration and use them for tracking

Once YOLACT training is complete, download the final weights file and the configuration file for future use. You train the network only once for a new class of object, and use the trained weights file in *Argos Tracking* tool for any video in which you want to track instances of this object. See the documentation for [Argos Tracking tool](https://argos.readthedocs.io/en/latest/user.html#track-interactively) to learn how to do this.